In [1]:
import pandas as pd
import regex as re
import os
from pathlib import Path

In [2]:
df = pd.read_csv('./DATASETS_excalidraw/file_lifecycle_events.csv', encoding='utf-8')
df.columns

Index(['file_path', 'status', 'commit_hash', 'commit_datetime',
       'commit_timestamp', 'author_name', 'author_email', 'commit_subject',
       'old_mode', 'new_mode', 'old_sha', 'new_sha', 'old_path', 'new_path',
       'source_path', 'similarity'],
      dtype='object')

In [3]:
df.sample()

,file_path,status,commit_hash,commit_datetime,commit_timestamp,author_name,author_email,commit_subject,old_mode,new_mode,old_sha,new_sha,old_path,new_path,source_path,similarity
12727,src/element/transformHandles.ts,modified,08ce7c7fc3e1e3056e349a6497014e3931c1578b,2022-08-03T15:28:17+00:00,1659540497,Aakansha Doshi,aakansha1216@gmail.com,feat: redesign linear elements 🎉 (#5501),100644,100644,f93f74d8a8ffcb9ee60faafd6f1cde4f9b8d50ba,715dcb3d7f053e09c8af9de5ce18c94ba3b8ef8d,NaN,NaN,NaN,NaN


In [ ]:
df[df['file_path'].apply(lambda x: '=>' in x)]['file_path'] # what are those arrows `=>` ??

ValueError: a must be greater than 0 unless no samples are taken